In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import requests

In [2]:
min_ = 10000
max_ = 10000
url = 'https://angel.co/companies?company_types[]=Startup&company_types[]=Private+Company&company_types[]=SaaS&company_types[]=Mobile+App&raised[min]='+str(min_)+'&raised[max]='+str(max_)

In [3]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

In [4]:
def load_pages():
    try:
        time.sleep(5)
        while True:
            button = driver.find_element_by_class_name('more')
            driver.execute_script("arguments[0].scrollIntoView(true)", button)
            button.click()
            time.sleep(1)
    except:
        pass

In [5]:
def get_startup_info():
    wanted_stuff = ['name', 'pitch', 'location','joined', 'website', 'market', 'company_size', 'stage', 'raised']
    all_startups = []
    startup_list = driver.find_elements_by_class_name('startup')
    for startup in startup_list:
        temp = {}
        for ws in wanted_stuff:
            temp[ws] = startup.find_element_by_class_name(ws).text
        all_startups.append(temp)
    return all_startups

In [6]:
def get_everything(n):
    min_ = 10000
    max_ = 10000
    base_url = 'https://angel.co/companies?company_types[]=Startup&company_types[]=Private+Company&company_types[]=SaaS&company_types[]=Mobile+App&raised[min]='
    url = base_url+str(min_)+'&raised[max]='+str(max_)
    driver.get(url)
    count = driver.find_element_by_xpath('//*[@id="root"]/div[4]/div[2]/div/div[2]/div[2]/div[1]/div[1]').text.split()[0]
    count = int(count)
    all_startups =[]
    while max_ < n:        
        print(len(all_startups))
        if count < 400:
            min_ = max_+10
            max_ = max_*1.2
            driver.get(base_url+str(min_)+'&raised[max]='+str(max_))
            count = driver.find_element_by_xpath('//*[@id="root"]/div[4]/div[2]/div/div[2]/div[2]/div[1]/div[1]').text.split()[0]
            count = int(count)
            if count > 0:
                time.sleep(2)
                load_pages()
                all_startups.extend(get_startup_info())
        elif count > 400:
            max_ = min_*1.05
            driver.get(base_url+str(min_)+'&raised[max]='+str(max_)+str(min_)+'&raised[max]='+str(max_))
            count = driver.find_element_by_xpath('//*[@id="root"]/div[4]/div[2]/div/div[2]/div[2]/div[1]/div[1]').text.split()[0]
            count = int(count)
    return all_startups
        

In [7]:
all_startups = get_everything(100000000000)

0
21
34
159
336
354
474
676
836
1436
1567
1722
1904
1964
1964
2204
2204
2424
2424
2593
2873
3270
3736
4352
4643
4877
5368
5528
5528
5928
5928
6328
6328
6368
6368
6888
7308
7308
7708
7708
8168
8168
8444
8707
9031
9344
9621
9870
10078
10252
10454
10621
10802
10962
11149
11293
11411
11626
11742
11845
11938
12043
12114
12185
12251
12303
12343
12381
12413
12433
12450
12470
12482
12493
12499
12499
12507
12514
12519
12522
12525
12527
12529
12531
12531
12532
12532
12533
12533
12533
12533
12533
12534
12535
12536
12536
12536
12536
12536
12536
12536
12536
12536
12536


In [10]:
df = pd.DataFrame(all_startups)
df.shape

(12536, 9)

In [4]:
df.head()

NameError: name 'df' is not defined

In [13]:
df.to_csv('all_startups.csv', encoding='utf-8')

In [18]:
df = pd.read_csv('all_startups.csv')
df.head()

,Unnamed: 0,company_size,joined,location,market,name,pitch,raised,stage,website
0,0,1-10,Dec ’13,Singapore,Big Data,enMarkit,Social Engagement and Business Intelligence Pl...,"$11,700",-,enmarkit.com
1,1,1-10,Feb ’14,Silicon Valley,Productivity Software,Limitless,Intelligent cross device continuity,"$12,000",NaN,angel.co
2,2,1-10,Jul ’12,Detroit,Video Streaming,Reel Deal,Building College Community Through Entertainme...,"$12,000",Seed,ReelDeal.tv
3,3,1-10,Jul ’13,Raleigh,Lifestyle,Walk [Your City],Online tools to drive offline mobility and hel...,"$11,364",Seed,walkyourcity.org
4,4,1-10,Mar ’14,Ann Arbor,Business Services,Savvy Languages,"Free, easy way to learn languages online","$12,000",Seed,savvylanguages.com


In [14]:
import os
api_key = os.environ['GMAP_API']

def get_coordinates(row):
    this_startup = {}
    place = row['location']
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    geo = requests.get(url,params=keys)
    geodata = geo.json()
    try:
        return pd.Series({'country' : geodata['results'][0]['address_components'][-4]['short_name'] , 'state' : geodata['results'][0]['address_components'][-3]['short_name']})
    except:
        print('Failed: ', geo.json())

In [17]:
place = 'Singapore'
url = 'https://maps.googleapis.com/maps/api/geocode/json'
keys = {'address': place, 'key': api_key}
geo = requests.get(url,params=keys)
geo.text

'{\n   "results" : [\n      {\n         "address_components" : [\n            {\n               "long_name" : "Singapore",\n               "short_name" : "SG",\n               "types" : [ "country", "political" ]\n            }\n         ],\n         "formatted_address" : "Singapore",\n         "geometry" : {\n            "bounds" : {\n               "northeast" : {\n                  "lat" : 1.4784001,\n                  "lng" : 104.0945001\n               },\n               "southwest" : {\n                  "lat" : 1.1496,\n                  "lng" : 103.594\n               }\n            },\n            "location" : {\n               "lat" : 1.352083,\n               "lng" : 103.819836\n            },\n            "location_type" : "APPROXIMATE",\n            "viewport" : {\n               "northeast" : {\n                  "lat" : 1.4784001,\n                  "lng" : 104.0945001\n               },\n               "southwest" : {\n                  "lat" : 1.1496,\n                

In [15]:
df.head().apply(get_coordinates, axis = 1)

Failed:  {'results': [{'address_components': [{'long_name': 'Singapore', 'short_name': 'SG', 'types': ['country', 'political']}], 'formatted_address': 'Singapore', 'geometry': {'bounds': {'northeast': {'lat': 1.4784001, 'lng': 104.0945001}, 'southwest': {'lat': 1.1496, 'lng': 103.594}}, 'location': {'lat': 1.352083, 'lng': 103.819836}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 1.4784001, 'lng': 104.0945001}, 'southwest': {'lat': 1.1496, 'lng': 103.594}}}, 'place_id': 'ChIJdZOLiiMR2jERxPWrUs9peIg', 'types': ['country', 'political']}], 'status': 'OK'}
Failed:  {'results': [{'address_components': [{'long_name': 'Silicon Valley', 'short_name': 'Silicon Valley', 'types': ['colloquial_area', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Silicon Valley, CA, USA', 'geometry': {'bounds': {'n

0                             None
1                             None
2          [Detroit, Wayne County]
3           [Raleigh, Wake County]
4    [Ann Arbor, Washtenaw County]
dtype: object